In [17]:
import requests
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
import plotly.express as px
import numpy as np
import json

In [24]:
numeric_vars = ['voyage_slaves_numbers__imp_total_num_slaves_disembarked',
          'voyage_slaves_numbers__imp_total_num_slaves_embarked']

geo_vars = ['voyage_itinerary__imp_principal_place_of_slave_purchase__value']

"""
geo_vars: lat, lon, port of emb
"""

"""
choose port of emb or disemb, i pack on the suffix which gets the lat and lon and the place name
"""
"""
dates_vars: start, end
"""

"""
call to value->pass to second url->get value->lookup on geo_codes->mutate df->then create bubble map: THIS IS THE WORKFLOW
"""

"""
port of emb, port of disemb, region, broad_region of emb and disemb
"""



"""
options call to voyages endpoint
load it as a dictionary
options_dict = options_dict[options[0]]
"""
r = requests.options('http://127.0.0.1:8000/voyage?hierarchical=False')
drop_dict = r.json()
drop_dict

{'id': {'type': "<class 'rest_framework.fields.IntegerField'>", 'label': 'ID'},
 'voyage_itinerary__id': {'type': "<class 'rest_framework.fields.IntegerField'>",
  'label': 'ID'},
 'voyage_itinerary__port_of_departure__id': {'type': "<class 'rest_framework.fields.IntegerField'>",
  'label': 'ID'},
 'voyage_itinerary__port_of_departure__place': {'type': "<class 'rest_framework.fields.CharField'>",
  'label': 'Port of departure (PORTDEP)'},
 'voyage_itinerary__port_of_departure__value': {'type': "<class 'rest_framework.fields.IntegerField'>",
  'label': 'Numeric code'},
 'voyage_itinerary__port_of_departure__longitude': {'type': "<class 'rest_framework.fields.DecimalField'>",
  'label': 'Longitude of point'},
 'voyage_itinerary__port_of_departure__latitude': {'type': "<class 'rest_framework.fields.DecimalField'>",
  'label': 'Latitude of point'},
 'voyage_itinerary__port_of_departure__show_on_main_map': {'type': "<class 'rest_framework.fields.BooleanField'>",
  'label': 'Show on main map

In [25]:
geo_drop = [drop_dict[i]['label'] for i in geo_vars]

emb = widgets.Dropdown(
    options=geo_drop,
    description="Parameter",
    disabled=False,
)
emb

Dropdown(description='Parameter', options=('Numeric code',), value='Numeric code')

In [3]:
"""
dropdown_options=[options_dict[i][‘label’] for i in numeric_vars]
"""
dropdown_option = [drop_dict[i]['label'] for i in numeric_vars]


"""
construct the url also from the values thrown from the dropdown
geo_vars : similar : array with two items, each of those items represent embarked disembarked
dates_vars : similar : ^
port of emb: lat, lon, name, num slaves emb
same for disemb
"""
"""
mutate df to include three new columns 
"""


"""
unclear as to why it is happening. http://127.0.0.1:8000/voyages/geo?hierarchical=False does not work
"""
"""
drop down says what the dates_vars the user wants: NUMERIC_DATES_VARS
then have a slider which chooses the dates as well
http://152.70.193.224:8000/voyage/aggregations?aggregate_fields=voyage_dates__imp_arrival_at_port_of_dis_yyyy
"""

"""
NUMERIC DATE VARS
voyage_dates__arrival_at_second_place_landing_yyyy
voyage_dates__date_departed_africa_yyyy
voyage_dates__departure_last_place_of_landing_yyyy
voyage_dates__first_dis_of_slaves_yyyy
voyage_dates__imp_arrival_at_port_of_dis_yyyy
voyage_dates__imp_departed_africa_yyyy
voyage_dates__imp_voyage_began_yyyy
voyage_dates__slave_purchase_began_yyyy
voyage_dates__third_dis_of_slaves_yyyy
voyage_dates__vessel_left_port_yyyy
voyage_dates__voyage_began_yyyy
voyage_dates__voyage_completed_yyyy
"""

emb = widgets.Dropdown(
    options=dropdown_option,
    description="Parameter",
    disabled=False,
)



emb

"""
dropdown will tell you the por
"""

"""
either build a dictionary which you can use to lookup the lat and lon or just put the lat lon in the DF
"""

Dropdown(description='Parameter', options=('Total slaves disembarked imputed * (SLAMIMP)', 'Total slaves embar…

In [14]:
url = 'http://152.70.193.224:8000/voyage/dataframes?selected_fields=voyage_dates__imp_arrival_at_port_of_dis_yyyy,voyage_itinerary__imp_principal_place_of_slave_purchase__latitude,voyage_itinerary__imp_principal_place_of_slave_purchase__longitude,voyage_itinerary__imp_principal_place_of_slave_purchase__place' + ',' + numeric_vars[0] + ',' + numeric_vars[1] + '&voyage_dates__imp_arrival_at_port_of_dis_yyyy=1810,1818'
URL2 = "http://152.70.193.224:8000/voyage/aggregations?aggregate_fields="
r = requests.get(url)
j = r.text
df = pd.read_json(j)
df = df.dropna()
df = df.sort_values(by=["voyage_dates__imp_arrival_at_port_of_dis_yyyy"])

In [5]:
df2 = df.groupby(['voyage_itinerary__imp_principal_place_of_slave_purchase__longitude', 
                  'voyage_itinerary__imp_principal_place_of_slave_purchase__latitude', 
                  'voyage_itinerary__imp_principal_place_of_slave_purchase__place', 
                  "voyage_dates__imp_arrival_at_port_of_dis_yyyy"], as_index=False).sum()

df2 = df2.rename({'voyage_itinerary__imp_principal_place_of_slave_purchase__longitude': 'Longitude',
                  'voyage_itinerary__imp_principal_place_of_slave_purchase__latitude': 'Latitude', 
                  'voyage_itinerary__imp_principal_place_of_slave_purchase__place':'Place of Purchase',
                  numeric_vars[0] : dropdown_option[0],
                  numeric_vars[1] : dropdown_option[1],
                  'voyage_dates__imp_arrival_at_port_of_dis_yyyy':'Year', 
                 }, axis=1)
df2

,Longitude,Latitude,Place of Purchase,Year,Total slaves disembarked imputed * (SLAMIMP),Total slaves embarked imputed * (slaximp)
0,-81.442000,30.687000,Amelia Island,1810,70.0,72.0
1,-81.442000,30.687000,Amelia Island,1811,3.0,4.0
2,-81.314700,29.894400,St. Augustine,1812,22.0,23.0
3,-77.396830,24.993640,New Providence,1811,290.0,295.0
4,-77.396830,24.993640,New Providence,1814,2.0,2.0
...,...,...,...,...,...,...
318,40.735760,-15.038460,Mozambique,1816,3508.0,3918.0
319,40.735760,-15.038460,Mozambique,1817,4113.0,5138.0
320,40.735760,-15.038460,Mozambique,1818,3815.0,4825.0
321,43.954497,-24.789244,Madagascar,1814,353.0,390.0


In [8]:
df2 = df2.assign(normalized_sizes = np.log(df2[emb.value]))
fig = px.scatter_geo(df2, lon = 'Longitude',
                     lat = 'Latitude',
                     hover_name = "Place of Purchase",
                     animation_frame = "Year",
                     size = 'normalized_sizes',
                     hover_data = [emb.value])
fig.show()

In [9]:
fig = px.bar(df2, x="Year",
             y=emb.value, color="Place of Purchase"
             )
fig.update_xaxes(dtick=1)
fig.show()

In [8]:
"""
Issues to discuss:
1. have to manually create generic labels
2. Had to use groupby as NaN values caused issues with graph creation
3. Unclear about port of emb and disemb and their relationship to the above data
"""

'\nIssues to discuss:\n1. have to manually create generic labels\n2. Had to use groupby as NaN values caused issues with graph creation\n3. Unclear about port of emb and disemb and their relationship to the above data\n'

In [18]:
print("+++FETCHING GEO+++")
r=requests.get("http://152.70.193.224:8000/voyage/geo?hierarchical=False&auto=True")
flatgeo=json.loads(r.text)
print("+++GEO FETCHED+++")

type_prefixes={'place':'','region':'region__','broad_region':'region__broad_region__'}
geo_codes={}
default_geocode=0

geo_codes[default_geocode]={
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [0,0]
    },
    "properties": {
        "name":"Unknown",
        "id":default_geocode
    }
}

for i in flatgeo:
    for tp in type_prefixes:
        prefix=type_prefixes[tp]
        geocode=i[prefix+'value']
        if geocode is None:
            geocode=default_geocode

        longitude=i[prefix+'longitude']
        latitude=i[prefix+'latitude']
        if latitude is None:
            latitude=0
        if longitude is None:
            longitude=0
        latitude=float(latitude)
        longitude=float(longitude)

        d={
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [longitude,latitude]
            },
            "properties": {
                "class":tp,
                tp:i[prefix+tp],
                "name":i[prefix+tp],
                "id":i[prefix+'value']
            }
        }
        for k in [j for j in i if j.startswith(prefix+"show_on")]:
            d['properties'][k]=i[k]
        geo_codes[geocode]=d
geo_codes

"""
classes: place, region, broad_region
loop through the entire dictionary 
"""

+++FETCHING GEO+++
+++GEO FETCHED+++


{0: {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [0, 0]},
  'properties': {'name': 'Unknown', 'id': 0}},
 10101: {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [0.0, 0.0]},
  'properties': {'class': 'place',
   'place': 'Alicante',
   'name': 'Alicante',
   'id': 10101,
   'show_on_main_map': False,
   'show_on_voyage_map': False}},
 10100: {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-6.862857, 37.200782]},
  'properties': {'class': 'region',
   'region': 'Spain',
   'name': 'Spain',
   'id': 10100,
   'region__show_on_map': True,
   'region__show_on_main_map': True}},
 10000: {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-8.561913, 37.213255]},
  'properties': {'class': 'broad_region',
   'broad_region': 'Europe',
   'name': 'Europe',
   'id': 10000,
   'region__broad_region__show_on_map': True}},
 10102: {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [2.18333, 41.38333]},
  'pr